In [1]:
import requests
import json
import pandas as pd
from pprint import pprint

In [2]:

#series_id = 1239527 
series_id = 1345038 # id for IPL 2023

#match_id = 1239543
match_id = 1359535 # G61, KKR v CSK
from_over = 0
inning_num = 1
url = f"https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId={series_id}&matchId={match_id}&inningNumber={inning_num}&commentType=ALL&fromInningOver={from_over}"
# resp = requests.get(url)
# data_json = json.loads(resp.text)
# data_json

In [3]:
def scrape_chunk(series_id, match_id, from_over, inning_num):
    """
    scrapes a chunk of commentary via AJAX request. The commentary comes in two over pieces, going backwards
    from the from_over parameter 
    e.g if from_over = 2 then we will get overs 1 and 2 which will have ball details looking like 0.1, 0.2, 1.1, 1.2 etc
    """
    url = f"https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId={series_id}&matchId={match_id}&inningNumber={inning_num}&commentType=ALL&fromInningOver={from_over}"
    #print(url)
    resp = requests.get(url)
    resp_json = json.loads(resp.text)
    chunk_data = resp_json['comments']
    return chunk_data

In [32]:

chunk = scrape_chunk(series_id, match_id, 1, 1)

print(type(chunk))
print(len(chunk))


<class 'list'>
6


In [33]:

pprint(type(chunk[0]))

for keys in chunk[0].keys():
    print(keys)
    


<class 'dict'>
_uid
id
inningNumber
ballsActual
ballsUnique
oversUnique
oversActual
overNumber
ballNumber
totalRuns
batsmanRuns
isFour
isSix
isWicket
dismissalType
byes
legbyes
wides
noballs
penalties
wagonZone
timestamp
modified
batsmanPlayerId
bowlerPlayerId
outPlayerId
totalInningRuns
totalInningWickets
predictions
title
dismissalText
commentPreTextItems
commentTextItems
commentPostTextItems
commentVideos
commentImages
events
over
batsmanStatText
bowlerStatText


In [34]:

pprint(chunk[0])


{'_uid': 42733372,
 'ballNumber': 6,
 'ballsActual': None,
 'ballsUnique': None,
 'batsmanPlayerId': 58403,
 'batsmanRuns': 0,
 'batsmanStatText': None,
 'bowlerPlayerId': 107007,
 'bowlerStatText': None,
 'byes': 0,
 'commentImages': [],
 'commentPostTextItems': None,
 'commentPreTextItems': None,
 'commentTextItems': [{'html': 'the first dot of the innings. Pitched up, '
                               'starts just outside off and swings away. '
                               'Conway has pushed this too well towards point',
                       'type': 'HTML'}],
 'commentVideos': [],
 'dismissalText': None,
 'dismissalType': None,
 'events': [],
 'id': 42733372,
 'inningNumber': 1,
 'isFour': False,
 'isSix': False,
 'isWicket': False,
 'legbyes': 0,
 'modified': '2023-05-14T14:04:39.000Z',
 'noballs': 0,
 'outPlayerId': None,
 'over': {'ballLimit': 120,
          'isComplete': True,
          'isMaiden': False,
          'isSuperOver': False,
          'overEndBatsmen': [{'balls': 

In [7]:

def scrape_innings(series_id, match_id, inning_num, debug=True):
    i = 2
    more_pages = True
    innings_data = []
    # test_data = scrape_chunk(series_id, match_id, 2, inning_num)
    # inning_exists = len(test_data) > 0
    if debug:
        print(f'scraping innings number {inning_num}')

    while more_pages:
        chunk_data = scrape_chunk(series_id, match_id, i, inning_num)

        # print(data_json['comments'][0]['oversActual'])
        if len(chunk_data) == 0:
            more_pages = False
            print('no more pages for inning')
        else:
            innings_data += reversed(chunk_data)
            # print(ball_data[0]['oversActual'])
            i += 2

    # if not inning_exists:
    #     return None
            
    return innings_data

In [8]:

def scrape_match(series_id, match_id):
    """
    Returns all ball-by-ball data for the given series and match id
    """
    innings = 1
    more_innings_exist = True
    match_data = []
    while more_innings_exist:
        innings_data = scrape_innings(series_id, match_id, inning_num=innings, debug=True)
        if len(innings_data) > 0:
            match_data += innings_data
            innings += 1
        else:
            print('innings does not exist')
            more_innings_exist = False
            
    return match_data


In [9]:
test = scrape_match(series_id, match_id)

scraping innings number 1
no more pages for inning
scraping innings number 2
no more pages for inning
scraping innings number 3
no more pages for inning
innings does not exist


In [20]:

print(type(test))
print(len(test))
print()
pprint(test[0])


<class 'list'>
243

{'_uid': 42733332,
 'ballNumber': 1,
 'ballsActual': None,
 'ballsUnique': None,
 'batsmanPlayerId': 95094,
 'batsmanRuns': 1,
 'batsmanStatText': None,
 'bowlerPlayerId': 107007,
 'bowlerStatText': None,
 'byes': 0,
 'commentImages': [],
 'commentPostTextItems': None,
 'commentPreTextItems': [{'html': '<strong>7.30pm</strong> Deep third is '
                                  'square. And there is a deep square leg. '
                                  'Here we go',
                          'type': 'HTML'},
                         {'html': '<strong>7.28pm</strong> The settled opening '
                                  'pair of Devon Conway and Ruturaj Gaikwad '
                                  'walks out for CSK. Vaibhav Arora will take '
                                  'the new ball.',
                          'type': 'HTML'},
                         {'html': '<strong>7.25pm</strong> Before we start, a '
                                  'word on the conditi

In [10]:
test[0]

{'_uid': 42733332,
 'id': 42733332,
 'inningNumber': 1,
 'ballsActual': None,
 'ballsUnique': None,
 'oversUnique': 0.01,
 'oversActual': 0.1,
 'overNumber': 1,
 'ballNumber': 1,
 'totalRuns': 1,
 'batsmanRuns': 1,
 'isFour': False,
 'isSix': False,
 'isWicket': False,
 'dismissalType': None,
 'byes': 0,
 'legbyes': 0,
 'wides': 0,
 'noballs': 0,
 'penalties': 0,
 'wagonZone': 4,
 'timestamp': '2023-05-14T14:00:46.000Z',
 'modified': '2023-05-14T14:00:49.000Z',
 'batsmanPlayerId': 95094,
 'bowlerPlayerId': 107007,
 'outPlayerId': None,
 'totalInningRuns': 1,
 'totalInningWickets': 0,
 'predictions': {'score': 169, 'winProbability': 47.48},
 'title': 'Arora to Gaikwad',
 'dismissalText': None,
 'commentPreTextItems': [{'type': 'HTML',
   'html': '<strong>7.30pm</strong> Deep third is square. And there is a deep square leg. Here we go'},
  {'type': 'HTML',
   'html': '<strong>7.28pm</strong> The settled opening pair of Devon Conway and Ruturaj Gaikwad walks out for CSK. Vaibhav Arora wil

In [11]:
temp = test[0]
temp

{'_uid': 42733332,
 'id': 42733332,
 'inningNumber': 1,
 'ballsActual': None,
 'ballsUnique': None,
 'oversUnique': 0.01,
 'oversActual': 0.1,
 'overNumber': 1,
 'ballNumber': 1,
 'totalRuns': 1,
 'batsmanRuns': 1,
 'isFour': False,
 'isSix': False,
 'isWicket': False,
 'dismissalType': None,
 'byes': 0,
 'legbyes': 0,
 'wides': 0,
 'noballs': 0,
 'penalties': 0,
 'wagonZone': 4,
 'timestamp': '2023-05-14T14:00:46.000Z',
 'modified': '2023-05-14T14:00:49.000Z',
 'batsmanPlayerId': 95094,
 'bowlerPlayerId': 107007,
 'outPlayerId': None,
 'totalInningRuns': 1,
 'totalInningWickets': 0,
 'predictions': {'score': 169, 'winProbability': 47.48},
 'title': 'Arora to Gaikwad',
 'dismissalText': None,
 'commentPreTextItems': [{'type': 'HTML',
   'html': '<strong>7.30pm</strong> Deep third is square. And there is a deep square leg. Here we go'},
  {'type': 'HTML',
   'html': '<strong>7.28pm</strong> The settled opening pair of Devon Conway and Ruturaj Gaikwad walks out for CSK. Vaibhav Arora wil

In [12]:
innings = 1
more_innings_exist = True
match_data = []
while more_innings_exist:
    innings_data = scrape_innings(series_id, match_id, inning_num=innings, debug=True)
    if len(innings_data) > 0:
        match_data += innings_data
        innings += 1
    else:
        print('innings does not exist')
        more_innings_exist = False


scraping innings number 1
no more pages for inning
scraping innings number 2
no more pages for inning
scraping innings number 3
no more pages for inning
innings does not exist


In [13]:
[ball['oversActual'] for ball in match_data][:500]

[0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 2.1,
 2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 3.1,
 3.2,
 3.3,
 3.4,
 3.5,
 3.6,
 4.1,
 4.2,
 4.3,
 4.4,
 4.5,
 4.6,
 5.1,
 5.2,
 5.3,
 5.4,
 5.5,
 5.6,
 6.1,
 6.2,
 6.3,
 6.4,
 6.5,
 6.6,
 7.1,
 7.2,
 7.3,
 7.4,
 7.5,
 7.6,
 8.1,
 8.2,
 8.3,
 8.4,
 8.5,
 8.6,
 9.1,
 9.2,
 9.3,
 9.4,
 9.5,
 9.6,
 10.1,
 10.2,
 10.3,
 10.4,
 10.5,
 10.6,
 11.1,
 11.2,
 11.3,
 11.4,
 11.5,
 11.6,
 12.1,
 12.2,
 12.3,
 12.4,
 12.5,
 12.6,
 13.1,
 13.2,
 13.3,
 13.4,
 13.5,
 13.6,
 14.1,
 14.2,
 14.3,
 14.4,
 14.5,
 14.6,
 15.1,
 15.2,
 15.3,
 15.4,
 15.5,
 15.6,
 16.1,
 16.2,
 16.3,
 16.4,
 16.4,
 16.5,
 16.6,
 17.1,
 17.2,
 17.3,
 17.4,
 17.4,
 17.5,
 17.6,
 18.1,
 18.2,
 18.3,
 18.4,
 18.5,
 18.5,
 18.6,
 19.1,
 19.2,
 19.3,
 19.4,
 19.5,
 19.5,
 19.5,
 19.6,
 0.1,
 0.2,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 1.1,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 2.1,
 2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 2.6,
 3.1,
 3.2,
 3.3,
 3.4,
 3.5,
 3.6,
 4.1,
 4.2,
 4.3,
 4.4,

In [16]:

print(type(innings_data))
print(len(innings_data))
print(innings_data[0])


<class 'list'>
0


IndexError: list index out of range